<a href="https://www.kaggle.com/code/ahmedalyafai/notebook9d51ddfc72?scriptVersionId=253277329" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        dir=os.path.join(dirname, filename)
        if(filename=="train.csv"):
            train_data=pd.read_csv(dir)
        elif(filename=="test.csv"):
            test_data=pd.read_csv(dir)
        elif(filename=="gender-submission.csv"):
            sample_s_data=pd.read_csv(dir)

### desscription of the numerical variables

In [ ]:
train_data.describe()

In [ ]:
train_data.describe(include='object')

In [ ]:
train_data.count()

In [ ]:
train_data.head()

# Data exploration

In [ ]:
df=train_data.copy()
Servived_grouped=df.groupby('Survived').agg(
    female_count=('Sex', lambda x: x[df.loc[x.index, 'Sex'] =='female'].count()),
    Male_count=('Sex', lambda x: x[df.loc[x.index, 'Sex']=='male'].count()),
    Fare=('Fare', 'mean')
)

Servived_grouped

In [ ]:
# Calculation the sum of null values
df.isnull().sum()

In [ ]:
df['Survived'].unique()

In [ ]:
df.groupby(['Survived','Sex'])['Age'].value_counts()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def visualize(x,y):
    sns.set_color_codes('pastel')
    sns.barplot(x=x, y=y, hue='Survived', data=df)
    plt.show()

In [ ]:
visualize('Sex', 'Age')

In [ ]:
visualize('Age', 'Embarked')

# Missing values exists!!
<b>in Cabin column where they can singificantly determines model's ability to predict<b>

<b>Therefore, we will do XGboost to predict the missing values for the Cabin then we will use as base along other variables to determines the missing values for the age column <b>

In [ ]:
df['Embarked'].unique()

In [ ]:
df.columns

In [ ]:
def counts(series):
    return series.value_counts().to_dict()
    
df.loc[~(df['Cabin'].isnull()), ['Pclass', 'Embarked', 'Fare','Cabin']].groupby(df['Embarked']).agg(
    Cabin_count=('Cabin',counts)
)

In [ ]:
# encode all categoircal columns at the same time
from sklearn.preprocessing import LabelEncoder
def label_encode(t):
    le=LabelEncoder()
    for i in t.columns:
        if i in t.select_dtypes(include='object'):
            t[i]=le.fit_transform(t[i])
    return t

In [ ]:
# Prepare the data for modelling
df2=df[~(df.isnull()).any(axis=1)].reset_index()
df2=label_encode(df2)
X=df2.drop(['Cabin', 'index', 'PassengerId', 'Name'], axis=1).copy()
Y=df2['Cabin'].copy()

# df3=df[df.isnull().any(axis=1)].reset_index()

In [ ]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_metrix
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test=train_test_split(X,Y, test_size=.3, random_state=42)

decision_tree=DecisionTreeClassifier(criterion='log_loss', splitter='best', 
                                           max_depth=None, min_samples_split=2)

decision_tree.fit(x_train, y_train)
# xgboost=XGBClassifier(objective='multi:softmax',n_estimators=1000, learning_rate=0.5, early_stopping_rounds=10)
# xgboost.fit(x_train, y_train,
#             verbose=True,
#             eval_set=[(x_test, y_test))
# y_predicted=xgboost.predict(x_test)
y_predicted=decision_tree.predict(x_test)
accuracy_score(y_test, y_predicted)


* Now we have to compute the missing values for the age column based on specific relationships

In [ ]:
#convert age field to numerical field
df['Age']=df['Age'].astype(int)

### The next Process involve:
making a correlation matrix to test the importance and the correlation between the variables


## prerocessing the data for correlation matrix

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
for i in df.columns:
    if i in df.select_dtypes(include='object'):
        df[i]=le.fit_transform(df[i])

In [ ]:
train_data.isnull().sum()

In [ ]:
import seaborn as sns

correlation_matrix=df.corr(method='pearson')
sns.heatmap(correlation_matrix, annot=True, fmt='.1f', linewidth=2)
plt.show()